In [25]:
#匯入所需模組
import os
import pandas as pd
import numpy as np
import cv2
import PIL.Image
from keras.preprocessing import image
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.applications import *
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten,MaxPooling2D
from keras.layers import Input,Dropout
from keras.models import Model

In [20]:
label_list=list()
img_name_list=list()
data_train=open('label.csv',"r")
for line in iter(data_train):
    img_name_list.append(line[:-5])
    if line[-2:-1]=="A":
        label_list.append(0)
    if line[-2:-1]=="B":
        label_list.append(1)
    if line[-2:-1]=="C":
        label_list.append(2)
data_train.close

<function TextIOWrapper.close()>

In [21]:
path='./sample_image/'
img_data_list=[]

for img in img_name_list:
    img_path=path+img
    photo=image.load_img(img_path,target_size=(224,224))
    g=image.img_to_array(photo)
    #x=np.expand_dims(x,axis=0)
    
    g=g/255.0
    img_data_list.append(g)
img_data=np.array(img_data_list)

In [22]:
all_classes=3
Y=np_utils.to_categorical(label_list,all_classes)
X,y=shuffle(img_data,Y,random_state=2)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=215)

In [27]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
import keras
model=Sequential()
image_input=Input(shape=(224,224,3))
model.add(VGG16(include_top=False, weights='imagenet', input_tensor=image_input, input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=256,activation='relu'))
model.add(Dense(all_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-5),metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=16,epochs=30,verbose=1)

Train on 75 samples, validate on 19 samples
Epoch 1/30


KeyboardInterrupt: 

In [ ]:

import keras
image_input=Input(shape=(299,299,3))
model=keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3),input_tensor=image_input)

tglast_layer = model.output
last_layer=Flatten()(last_layer)
last_layer=Dropout(0.5)(last_layer)
out = Dense(all_classes, activation='softmax', name='softmax')(last_layer)
custom_V3_model = Model(image_input, out)
# custom_V3_model.summary()

custom_V3_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-5),metrics=['accuracy'])
custom_V3_model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=16,epochs=30,verbose=1)


In [ ]:
prediction=custom_V3_model.predict(X_test)
predict=np.argmax(prediction,axis=1)

In [ ]:
from sklearn.metrics import recall_score

y_true=np.argmax(y_test,axis=1)
recall_score(y_true, predict, average='weighted')